# <center><font color = '#DF9166' size = 20 center> **Data Auto Labeling**</font></center>

## <font color = '#DF9166' size=6>**Table of content**<font/><a class = 'anchor' id = 'introduction'/>

1. [**Import Libraries**](#import)
2. [**Data Loading**](#data_loading)
3. [**Data Inspection**](#data_inspection)
4. [**Extract Location**](#extract_location)
5. [**Extract Price**](#extract_price)
6. [**Extract Product**](#extract_product)
7. [**Generate CoNLL Label**](#generate_conll)

## <font color = '#DF9166' size=6>**Import Libraries**<font/><a class = 'anchor' id = 'import'/>


In [1]:
import os
import sys
import warnings
import pandas as pd

In [2]:
warnings.filterwarnings("ignore")
pd.set_option("display.max_colwidth", None)

In [ ]:
sys.path.append(os.path.abspath(os.pardir))
from scripts.label_data import *

## <font color = '#DF9166' size=6>**Data Loading**<font/><a class = 'anchor' id = 'data_loading'/>

In [7]:
DATA_PATH = "../data/processed/processed_data.csv"

In [8]:
df = pd.read_csv(DATA_PATH, index_col=0)
df.sample(5)

,channel_name,channel_address,channel_id,message_id,date,message,message_length,word_count,cleaned_message
28461,EthioBrand®,@ethio_brand_collection,1149977975,322,2017-09-06 07:58:37,Adidas\nmade in vietnam\nprice 1250 birr Contact me @Sofonias12 or call the number in the picture for more info,109,20,Adidas\nmade in vietnam\nprice 1250 birr Contact me @Sofonias12 or call the number in the picture for more info
23963,ምርጥ ዕቃ,@MerttEka,1370292088,2504,2022-09-29 16:39:18,BURGER PRESS\n🎯 ከአልሙኒየም የተሰራ\n🎯ጠንካራ \n🎯 አጠቃቀሙ ቀላል\n👉 400 ብር\n\n☎️ 0904944848\n☎️ 0911928738\n\n@ordermertteka1\n@ordermertteka2\n👉መገናኛ ዘፍመሽ ግራንድ ሞል 3ኛ ፎቅ ከሊፍት ሲወርዱ ወደ ቀኝ ታጥፈው 376,167,30,BURGER PRESS\n ከአልሙኒየም የተሰራ\nጠንካራ \n አጠቃቀሙ ቀላል\n 400 ብር\n\n 0904944848\n 0911928738\n\n@ordermertteka1\n@ordermertteka2\nመገናኛ ዘፍመሽ ግራንድ ሞል 3ኛ ፎቅ ከሊፍት ሲወርዱ ወደ ቀኝ ታጥፈው 376
52405,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,1424695758,4486,2022-02-23 19:28:39,ኳሊቲ ወፋራም የሴት ልጆች ሹራብ\n1000 ብር\nከ6 -14 አመት\n0945097042\n0905707448,61,12,ኳሊቲ ወፋራም የሴት ልጆች ሹራብ\n1000 ብር\nከ6 -14 አመት\n0945097042\n0905707448
20530,NEVA COMPUTER®,@nevacomputer,1195361398,7050,2022-12-25 21:02:19,"🌟🌟NEW ARRIVAL!🌟🌟\n👉 hp elitebook \n⭐️ intel®CORE i7 💪6th generation\nModel : elitebook 840\n💎GRAPHICS: intel®hd graphics\n🖥 Screen :14"" inch FULL HD \n📼 Storage :256GB SSD\n⏳Ram : 8 DDR4\n🔋: 5hours ++ battery life\n\nPrice👇\n📩 @rasneva ለአጭር መልእክት\n\n+251912759900\n+251920153333\n\n🏢 አድራሻ : - መገናኛ ማራቶን የ ገበያ ማእከል በ ዋናው መግቢያ መሬት ላይ\nወይንም ግራውንድ ፍሎር ብቅ ይበሉ\n 👉ነቫ ኮምፒውተር👈 መሆኑን ያረጋግጡ\n\nድህረ ገጻችንን ይጎብኙ\nwww.nevacomputer.com\n\nቴሌግራም ቻናላችንን ይቀላቀሉ\n\nhttps://t.me/nevacomputer",445,74,"NEW ARRIVAL!\n hp elitebook \n intelCORE i7 6th generation\nModel : elitebook 840\nGRAPHICS: intelhd graphics\n Screen :14"" inch FULL HD \n Storage :256GB SSD\nRam : 8 DDR4\n: 5hours ++ battery life\n\nPrice\n @rasneva ለአጭር መልእክት\n\n+251912759900\n+251920153333\n\n አድራሻ : - መገናኛ ማራቶን የ ገበያ ማእከል በ ዋናው መግቢያ መሬት ላይ\nወይንም ግራውንድ ፍሎር ብቅ ይበሉ\n ነቫ ኮምፒውተር መሆኑን ያረጋግጡ\n\nድህረ ገጻችንን ይጎብኙ\nwww.nevacomputer.com\n\nቴሌግራም ቻናላችንን ይቀላቀሉ\n\nhttps://t.me/nevacomputer"
33852,ማራኪ ცЯﾑŋの™,@marakibrand,1320403852,5133,2020-09-21 20:00:41,"♕ Men's Fashion \n🏷️ Price: 1400 ETB\n 📐Size: Available \n📩 INFO: @Bilen2211\n✆ Mobile: 0933682917\n┃Free Delivery┃ \n ✄〰〰〰〰〰〰〰〰〰〰\n🚩 አድራሻ - አዲስ አበባ, ሜክሲኮ፡ ከ ኬኬር ህንጻ 50ሜ ወረድ ብሎ አይመን ህንፃ ግራውንድ ፍሎር ላይ፡ የሱቅ ቁ.012\n⊰━━━━━━━⊱⊰━━━━━━━⊰\nMaraki Brand™ ┃ማራኪ ብራንድ™ \n⊰━━━━━━━⊱⊰━━━━━━━⊱",268,43,"Men's Fashion \n Price: 1400 ETB\n Size: Available \n INFO: @Bilen2211\n Mobile: 0933682917\nFree Delivery \n \n አድራሻ - አዲስ አበባ, ሜክሲኮ፡ ከ ኬኬር ህንጻ 50ሜ ወረድ ብሎ አይመን ህንፃ ግራውንድ ፍሎር ላይ፡ የሱቅ ቁ.012\n\nMaraki Brand ማራኪ ብራንድ \n"


## <font color = '#DF9166' size=6>**Data Inspection**<font/><a class = 'anchor' id = 'data_inspection'/>

In [9]:
channel_addresses = df.channel_address.unique().tolist()
print(channel_addresses)

['@qnashcom', '@ZemenExpress', '@classybrands', '@nevacomputer', '@MerttEka', '@ethio_brand_collection', '@marakibrand', '@helloomarketethiopia', '@marakisat2', '@belaclassic', '@Shewabrand', '@sinayelj', '@kuruwear', '@gebeyaadama', '@modernshoppingcenter', '@aradabrand2', '@Leyueqa', '@AwasMart', '@meneshayeofficial', '@Fashiontera', '@forfreemarket']


In [10]:
channel_address = channel_addresses[0]
# Set the random seed for reproducibility
seed_value = 42

# Sample 15 messages from '@qnashcom' and clean the 'message' column
sampled_df = (
    df[df["channel_address"] == channel_address]
    .sample(n=15, random_state=seed_value)
    .reset_index(drop=True)
)
sampled_df.head(2)

,channel_name,channel_address,channel_id,message_id,date,message,message_length,word_count,cleaned_message
0,qnash.com - ቅናሽ ®️,@qnashcom,1336267238,2458,2022-12-29 17:58:51,💯🤩 ኪችንዎን በቅጽበት ውብ🤩 ማድረግ ይፈልጋሉ?\n💁‍♀ለኪችንዎ ግርማ ሞገስ✨\n💥' ሀርቪ ስቲከር '💥\n✅ ለመሳቢያ ፣ ለካቢኔ ፣ ለኩሽና ፣ ለስላሳ የእንጨት ገጽና ግድግዳ ላይ የሚለጠፍ ማስጌጫ\n✅ በራሱ የሚጣበቅ📎\nውሃ የማያሳልፍ (Waterproof)💯\n✅ ሙቀትን🔥 እና እርጥበትን 🚿የሚቋቋም\n✅ ለማፅዳት ቀላል የሆነ\n✨✨አንጸባራቂ✨✨\n\n❇️ Kitchen Sticker\nTelegram - t.me/qnashcom\n👉Size: 60CM×5M\n🌀Water-proof \n🌀Oil-proof \n🌀Hot-proof \n\n 💰ዋጋ:- 5 ሜትር :- ✅ 800 ብር\n\nውስን ፍሬ ነው የቀረው ❌❌\n\n\nTelegram - t.me/qnashcom\nWebsite - www.qnash.com\n✅ ጥራት ✅ ዋስትና ✅ ቅናሽ\n\n🏬 አድራሻ ፦ \nቁጥር 1 :- መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 \n\nቁጥር 2 :- ጀሞ 1 ከለላ ህንፃ ግራውንድ ለይ G07 \n\n❌❌ ማሳሰብያ: ሱቃችን ሲመጡ ትክክለኛ የኛ ሱቅ መሆኑ ያረጋግጡ የራሳችን ሎጎ መኖሩን እና 329 / G07 መሆኑ ያረጋግጡ !\n\nስልክ:\n📱0946966440\n📱0992606060\n📱0905464599\n☎ +251118639952,656,128,ኪችንዎን በቅጽበት ውብ ማድረግ ይፈልጋሉ?\nለኪችንዎ ግርማ ሞገስ\n' ሀርቪ ስቲከር '\n ለመሳቢያ ፣ ለካቢኔ ፣ ለኩሽና ፣ ለስላሳ የእንጨት ገጽና ግድግዳ ላይ የሚለጠፍ ማስጌጫ\n በራሱ የሚጣበቅ\nውሃ የማያሳልፍ (Waterproof)\n ሙቀትን እና እርጥበትን የሚቋቋም\n ለማፅዳት ቀላል የሆነ\nአንጸባራቂ\n\n Kitchen Sticker\nTelegram - t.me/qnashcom\nSize: 60CM5M\nWater-proof \nOil-proof \nHot-proof \n\n ዋጋ:- 5 ሜትር :- 800 ብር\n\nውስን ፍሬ ነው የቀረው \n\n\nTelegram - t.me/qnashcom\nWebsite - www.qnash.com\n ጥራት ዋስትና ቅናሽ\n\n አድራሻ ፦ \nቁጥር 1 :- መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 \n\nቁጥር 2 :- ጀሞ 1 ከለላ ህንፃ ግራውንድ ለይ G07 \n\n ማሳሰብያ: ሱቃችን ሲመጡ ትክክለኛ የኛ ሱቅ መሆኑ ያረጋግጡ የራሳችን ሎጎ መኖሩን እና 329 / G07 መሆኑ ያረጋግጡ !\n\nስልክ:\n0946966440\n0992606060\n0905464599\n +251118639952
1,qnash.com - ቅናሽ ®️,@qnashcom,1336267238,612,2021-09-01 20:51:25,"Wild Tornado\n➡️Sink and Drain Cleaner \nለኩሽና, ለሽንት ቤት , ለሲንክ, ለሻውር ቤት, ወዘተ.....\n\n💰ዋጋ✅ 550 ብር \n\n🏬 አድራሻ መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329\nስልክ:\n📱0946966440\n📱0905464599\n☎ +251118639952\n🚚 ያሉበት በነፃ እናመጣለን 🏍️",201,38,"Wild Tornado\nSink and Drain Cleaner \nለኩሽና, ለሽንት ቤት , ለሲንክ, ለሻውር ቤት, ወዘተ.....\n\nዋጋ 550 ብር \n\n አድራሻ መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329\nስልክ:\n0946966440\n0905464599\n +251118639952\n ያሉበት በነፃ እናመጣለን"


In [11]:
sampled_messages = sampled_df["cleaned_message"]

for idx, sampled_message in enumerate(sampled_messages):
    print("_" * 50, f"\nSample {idx+1}\n", "_" * 50, sep="")
    print(sampled_message)

__________________________________________________
Sample 1
__________________________________________________
 ኪችንዎን በቅጽበት ውብ ማድረግ ይፈልጋሉ?
ለኪችንዎ ግርማ ሞገስ
' ሀርቪ ስቲከር '
 ለመሳቢያ ፣ ለካቢኔ ፣ ለኩሽና ፣ ለስላሳ የእንጨት ገጽና ግድግዳ ላይ የሚለጠፍ ማስጌጫ
 በራሱ የሚጣበቅ
ውሃ የማያሳልፍ (Waterproof)
 ሙቀትን እና እርጥበትን የሚቋቋም
 ለማፅዳት ቀላል የሆነ
አንጸባራቂ

 Kitchen Sticker
Telegram - t.me/qnashcom
Size: 60CM5M
Water-proof 
Oil-proof 
Hot-proof 

 ዋጋ:-  5 ሜትር :-  800 ብር

ውስን ፍሬ ነው የቀረው 


Telegram - t.me/qnashcom
Website - www.qnash.com
 ጥራት  ዋስትና  ቅናሽ

 አድራሻ ፦ 
ቁጥር 1 :- መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 

ቁጥር 2 :- ጀሞ 1 ከለላ ህንፃ ግራውንድ ለይ G07 

 ማሳሰብያ: ሱቃችን ሲመጡ ትክክለኛ የኛ ሱቅ መሆኑ ያረጋግጡ  የራሳችን ሎጎ መኖሩን እና 329 / G07 መሆኑ ያረጋግጡ !

ስልክ:
0946966440
0992606060
0905464599
 +251118639952
__________________________________________________
Sample 2
__________________________________________________
Wild Tornado
Sink and Drain Cleaner 
ለኩሽና, ለሽንት ቤት , ለሲንክ, ለሻውር ቤት, ወዘተ.....

ዋጋ 550 ብር 

 አድራሻ መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329
ስልክ:
0946966440
0905464599
 +2511

## <font color = '#DF9166' size=6>**Extract Location**<font/><a class = 'anchor' id = 'extract_location'/>

**Location Extraction Pattern:**  

- Extracts location details that start with **"አድራሻ"** (Address) or **"ቁጥር X :-"** (where X is a number).  
- Captures all text following these keywords until the next **newline (`\n`) or end of text (`$`)**.  
- Supports locations with multiple lines by joining them into a single string.  
- Retains the original text format without modifications.  
- Returns `None` if no matching location pattern is found.  

In [13]:
sampled_df["location"] = sampled_df["cleaned_message"].apply(extract_location)
sampled_df[["cleaned_message", "location"]].head()

,cleaned_message,location
0,ኪችንዎን በቅጽበት ውብ ማድረግ ይፈልጋሉ?\nለኪችንዎ ግርማ ሞገስ\n' ሀርቪ ስቲከር '\n ለመሳቢያ ፣ ለካቢኔ ፣ ለኩሽና ፣ ለስላሳ የእንጨት ገጽና ግድግዳ ላይ የሚለጠፍ ማስጌጫ\n በራሱ የሚጣበቅ\nውሃ የማያሳልፍ (Waterproof)\n ሙቀትን እና እርጥበትን የሚቋቋም\n ለማፅዳት ቀላል የሆነ\nአንጸባራቂ\n\n Kitchen Sticker\nTelegram - t.me/qnashcom\nSize: 60CM5M\nWater-proof \nOil-proof \nHot-proof \n\n ዋጋ:- 5 ሜትር :- 800 ብር\n\nውስን ፍሬ ነው የቀረው \n\n\nTelegram - t.me/qnashcom\nWebsite - www.qnash.com\n ጥራት ዋስትና ቅናሽ\n\n አድራሻ ፦ \nቁጥር 1 :- መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 \n\nቁጥር 2 :- ጀሞ 1 ከለላ ህንፃ ግራውንድ ለይ G07 \n\n ማሳሰብያ: ሱቃችን ሲመጡ ትክክለኛ የኛ ሱቅ መሆኑ ያረጋግጡ የራሳችን ሎጎ መኖሩን እና 329 / G07 መሆኑ ያረጋግጡ !\n\nስልክ:\n0946966440\n0992606060\n0905464599\n +251118639952,አድራሻ ፦ ቁጥር 1 :- መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 ቁጥር 2 :- ጀሞ 1 ከለላ ህንፃ ግራውንድ ለይ G07
1,"Wild Tornado\nSink and Drain Cleaner \nለኩሽና, ለሽንት ቤት , ለሲንክ, ለሻውር ቤት, ወዘተ.....\n\nዋጋ 550 ብር \n\n አድራሻ መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329\nስልክ:\n0946966440\n0905464599\n +251118639952\n ያሉበት በነፃ እናመጣለን",አድራሻ መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329
2,"Hair Dye Brush Bottle\n\nMade of quality plastic material\nGreat tool for hair dyeing use\nEasy to mix hair color potion\nSuitable for both professional or home DIY use\nAttached with a comb, durable and safe to use\nBottle with scale, easy to mix hair color potion\n\nCapacity: 160ml\n\n ዋጋ፦ 250 ብር \n\nውስን ፍሬ ነው የቀረው \n< Limited Stock \n\nTelegram - t.me/qnashcom\nWebsite - www.qnash.com\n ጥራት ዋስትና ቅናሽ\n አድራሻ መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329\nስልክ:\n0946966440\n0905464599\n +251118639952",አድራሻ መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329
3,Door Bottom Seal Strip Stopper \nKeep Hot and Cold Air isolated \n Reduce The Noise Outside and Door slamming \n No \n \nSize 93cm * 10cm \n\n 300 ብር \n\nTelegram - t.me/qnashcom\nWebsite - www.qnash.com\n ጥራት ዋስትና ቅናሽ\n አድራሻ መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329\nስልክ:\n0946966440\n0905464599\n +251118639952,አድራሻ መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329
4,"Washing Machine Anti-Vibration Feet Pads (Shock Pad)\n\n4Packs Anti-Vibration Pads for Washing Machine Home Appliances, Silent Anti-Walk Feet Pads Washer and Dryer Noise Cancelling Foot Pads for All Washing Machine\n\n ዋጋ፦ 350 ብር\n\nውስን ፍሬ ነው የቀረው \n\nTelegram - t.me/qnashcom\nWebsite - www.qnash.com\n ጥራት ዋስትና ቅናሽ\n አድራሻ መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 ከሊፍት እንደወረዱ በግራ በኩል የፎቁ መጨረሻ ዞር እንዳሉ\nስልክ:\n0946966440\n0905464599\n +251118639952",አድራሻ መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 ከሊፍት እንደወረዱ በግራ በኩል የፎቁ መጨረሻ ዞር እንዳሉ


## <font color = '#DF9166' size=6>**Extract Price**<font/><a class = 'anchor' id = 'extract_price'/>

**Price Extraction Pattern:**  

- Extracts price values from text that include **"ዋጋ"** (Price) and **"ብር"** (Ethiopian currency).  
- Supports different formats, such as:  
  - **With "ዋጋ"** → *"ዋጋ፦ 250 ብር"*, *"ዋጋ:- 5 ሜትር :- 800 ብር"*.  
  - **Without "ዋጋ"** → *"300 ብር"*, *"1100 ብር"*.  
  - **Comma-separated values** → *"1,200 ብር"*.  
  - **Multiple prices in one sentence** → *"ዋጋ፦ 950 800 ብር"*.  
- Matches **one or two consecutive numbers** before "ብር" (e.g., *"950 800 ብር"*).  
- Returns extracted prices as a **comma-separated string**, or `None` if no price is found.  

In [14]:
sampled_df["price"] = sampled_df["cleaned_message"].apply(extract_price)
sampled_df[["cleaned_message", "price"]]

,cleaned_message,price
0,ኪችንዎን በቅጽበት ውብ ማድረግ ይፈልጋሉ?\nለኪችንዎ ግርማ ሞገስ\n' ሀርቪ ስቲከር '\n ለመሳቢያ ፣ ለካቢኔ ፣ ለኩሽና ፣ ለስላሳ የእንጨት ገጽና ግድግዳ ላይ የሚለጠፍ ማስጌጫ\n በራሱ የሚጣበቅ\nውሃ የማያሳልፍ (Waterproof)\n ሙቀትን እና እርጥበትን የሚቋቋም\n ለማፅዳት ቀላል የሆነ\nአንጸባራቂ\n\n Kitchen Sticker\nTelegram - t.me/qnashcom\nSize: 60CM5M\nWater-proof \nOil-proof \nHot-proof \n\n ዋጋ:- 5 ሜትር :- 800 ብር\n\nውስን ፍሬ ነው የቀረው \n\n\nTelegram - t.me/qnashcom\nWebsite - www.qnash.com\n ጥራት ዋስትና ቅናሽ\n\n አድራሻ ፦ \nቁጥር 1 :- መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 \n\nቁጥር 2 :- ጀሞ 1 ከለላ ህንፃ ግራውንድ ለይ G07 \n\n ማሳሰብያ: ሱቃችን ሲመጡ ትክክለኛ የኛ ሱቅ መሆኑ ያረጋግጡ የራሳችን ሎጎ መኖሩን እና 329 / G07 መሆኑ ያረጋግጡ !\n\nስልክ:\n0946966440\n0992606060\n0905464599\n +251118639952,800 ብር
1,"Wild Tornado\nSink and Drain Cleaner \nለኩሽና, ለሽንት ቤት , ለሲንክ, ለሻውር ቤት, ወዘተ.....\n\nዋጋ 550 ብር \n\n አድራሻ መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329\nስልክ:\n0946966440\n0905464599\n +251118639952\n ያሉበት በነፃ እናመጣለን",ዋጋ 550 ብር
2,"Hair Dye Brush Bottle\n\nMade of quality plastic material\nGreat tool for hair dyeing use\nEasy to mix hair color potion\nSuitable for both professional or home DIY use\nAttached with a comb, durable and safe to use\nBottle with scale, easy to mix hair color potion\n\nCapacity: 160ml\n\n ዋጋ፦ 250 ብር \n\nውስን ፍሬ ነው የቀረው \n< Limited Stock \n\nTelegram - t.me/qnashcom\nWebsite - www.qnash.com\n ጥራት ዋስትና ቅናሽ\n አድራሻ መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329\nስልክ:\n0946966440\n0905464599\n +251118639952",ዋጋ፦ 250 ብር
3,Door Bottom Seal Strip Stopper \nKeep Hot and Cold Air isolated \n Reduce The Noise Outside and Door slamming \n No \n \nSize 93cm * 10cm \n\n 300 ብር \n\nTelegram - t.me/qnashcom\nWebsite - www.qnash.com\n ጥራት ዋስትና ቅናሽ\n አድራሻ መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329\nስልክ:\n0946966440\n0905464599\n +251118639952,300 ብር
4,"Washing Machine Anti-Vibration Feet Pads (Shock Pad)\n\n4Packs Anti-Vibration Pads for Washing Machine Home Appliances, Silent Anti-Walk Feet Pads Washer and Dryer Noise Cancelling Foot Pads for All Washing Machine\n\n ዋጋ፦ 350 ብር\n\nውስን ፍሬ ነው የቀረው \n\nTelegram - t.me/qnashcom\nWebsite - www.qnash.com\n ጥራት ዋስትና ቅናሽ\n አድራሻ መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 ከሊፍት እንደወረዱ በግራ በኩል የፎቁ መጨረሻ ዞር እንዳሉ\nስልክ:\n0946966440\n0905464599\n +251118639952",ዋጋ፦ 350 ብር
5,Helping Handle\n\nHelping Handle with Strong Sucker Hand Grip Handrail to Keep Balance for Bedroom Bath Room Bathroom\n\n ዋጋ፦ 700 ብር\n\nውስን ፍሬ ነው የቀረው \nLimited stock \n\nTelegram - t.me/qnashcom\n ጥራት ዋስትና ቅናሽ\n\n አድራሻ ፦ \nቁጥር 1 :- መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 \n\nቁጥር 2 :- ጀሞ 1 ከለላ ህንፃ ግራውንድ ለይ G07 \n\n ማሳሰብያ: ሱቃችን ሲመጡ ትክክለኛ የኛ ሱቅ መሆኑ ያረጋግጡ የራሳችን ሎጎ መኖሩን እና 329 / G07 መሆኑ ያረጋግጡ !\n\nስልክ:\n 0946966440\n 0992606060\n 0905464599\n +251118639952,ዋጋ፦ 700 ብር
6,ICE Cube Maker Genie\n የበረዶ መስሪያ \n\nKeep Ice-cream cold \nKeep drinks Ice-cold \nEasy to use \nSave a lot of space in your freezer; \nKeeps your cubes fresh and odor free with air tight lid;\n\n ዋጋ፦ 600 ብር \n\nውስን ፍሬ ነው የቀረው \n< Limited Stock \n\nTelegram - t.me/qnashcom\nጥራት ዋስትና ቅናሽ\n\n አድራሻ ፦ \nቁጥር 1 :- መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 \n\nቁጥር 2 :- ጀሞ 1 ከለላ ህንፃ ግራውንድ ለይ G07 \n\n ማሳሰብያ: ሱቃችን ሲመጡ ትክክለኛ የኛ ሱቅ መሆኑ ያረጋግጡ የራሳችን ሎጎ መኖሩን እና 329 / G07 መሆኑ ያረጋግጡ !\n\nስልክ:\n0946966440\n0992606060\n0905464599\n +251118639952,ዋጋ፦ 600 ብር
7,Automatic Rebound Healthy Abdomen Wheel\n\nAutomatic Rebound Abdominal Wheel for Core Trainer Workout \n Ab Workout Equipment with Elbow Support \nHome Abdominal Exercise Machine \nand Plank Roller in One\n\n የቦርጭ ስብን ለማቅለጥ እና ፓክ ለማውጣት የሚመረጥ\n በቀላሉ የሆድ፣ የመቀመጫ፣ የጀርባ ጡንቻዎትን ለማሳሳብና ለማጠንከር ይረዳዎታል\n የራሱ መቁጠሪያ ሰዓት የተካተተለት\n በቀላሉ የተሸናቀጠ ተክለሰውነትን ይላበሱ\n\nReliable & Durable\nEasy & Comfortable \nEasy to use \nVast Application \nExcellent Gift Alternative\n\n ዋጋ፦ 2600 ብር \n\n ለማዘዝ = @Orderqnash\n\n ስለ ዕቃወቹ መረጃ እና ቪዲዮዎች ለማየት ቴሌግራማንን ይቀላቀሉ\nTelegram - t.me/qnashcom\n\nውስን ፍሬ ነው የቀረው \nLimited Stock \n ይፍጠኑ \n\nTelegram - t.me/qnashcom\nጥራት ዋስትና ቅናሽ\n\n አድራሻ ፦ \nቁጥር 1 :- መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 \n\nቁጥር 2 :- ጀሞ 1 ከለላ ህንፃ ግራውንድ ለይ G07 \n\n ማሳሰብያ: ሱቃችን ሲመጡ ትክክለኛ የኛ ሱቅ መሆኑ ያረጋግጡ የራሳችን ሎጎ መኖሩን እና 329 / G07 መሆኑ ያረጋግጡ !\n\nስ

## <font color = '#DF9166' size=6>**Extract Product**<font/><a class = 'anchor' id = 'extract_product'/>


**Product Extraction Pattern:**  

- Extracts the **first one or two lines** of a message as the product description.  
- The **first line**:  
  - Must contain text (Amharic or English).  
  - **Cannot** include `!` or `?` (other punctuation marks are allowed).  
- The **second line** (if included):  
  - Must contain **only Amharic text** (no English or punctuation).  
  - If it contains only spaces or punctuation, it is ignored.  
- Returns `None` if the conditions are not met.

In [15]:
sampled_df["product"] = sampled_df["cleaned_message"].apply(extract_product)
sampled_df[["cleaned_message", "product"]]

,cleaned_message,product
0,ኪችንዎን በቅጽበት ውብ ማድረግ ይፈልጋሉ?\nለኪችንዎ ግርማ ሞገስ\n' ሀርቪ ስቲከር '\n ለመሳቢያ ፣ ለካቢኔ ፣ ለኩሽና ፣ ለስላሳ የእንጨት ገጽና ግድግዳ ላይ የሚለጠፍ ማስጌጫ\n በራሱ የሚጣበቅ\nውሃ የማያሳልፍ (Waterproof)\n ሙቀትን እና እርጥበትን የሚቋቋም\n ለማፅዳት ቀላል የሆነ\nአንጸባራቂ\n\n Kitchen Sticker\nTelegram - t.me/qnashcom\nSize: 60CM5M\nWater-proof \nOil-proof \nHot-proof \n\n ዋጋ:- 5 ሜትር :- 800 ብር\n\nውስን ፍሬ ነው የቀረው \n\n\nTelegram - t.me/qnashcom\nWebsite - www.qnash.com\n ጥራት ዋስትና ቅናሽ\n\n አድራሻ ፦ \nቁጥር 1 :- መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 \n\nቁጥር 2 :- ጀሞ 1 ከለላ ህንፃ ግራውንድ ለይ G07 \n\n ማሳሰብያ: ሱቃችን ሲመጡ ትክክለኛ የኛ ሱቅ መሆኑ ያረጋግጡ የራሳችን ሎጎ መኖሩን እና 329 / G07 መሆኑ ያረጋግጡ !\n\nስልክ:\n0946966440\n0992606060\n0905464599\n +251118639952,None
1,"Wild Tornado\nSink and Drain Cleaner \nለኩሽና, ለሽንት ቤት , ለሲንክ, ለሻውር ቤት, ወዘተ.....\n\nዋጋ 550 ብር \n\n አድራሻ መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329\nስልክ:\n0946966440\n0905464599\n +251118639952\n ያሉበት በነፃ እናመጣለን",Wild Tornado
2,"Hair Dye Brush Bottle\n\nMade of quality plastic material\nGreat tool for hair dyeing use\nEasy to mix hair color potion\nSuitable for both professional or home DIY use\nAttached with a comb, durable and safe to use\nBottle with scale, easy to mix hair color potion\n\nCapacity: 160ml\n\n ዋጋ፦ 250 ብር \n\nውስን ፍሬ ነው የቀረው \n< Limited Stock \n\nTelegram - t.me/qnashcom\nWebsite - www.qnash.com\n ጥራት ዋስትና ቅናሽ\n አድራሻ መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329\nስልክ:\n0946966440\n0905464599\n +251118639952",Hair Dye Brush Bottle
3,Door Bottom Seal Strip Stopper \nKeep Hot and Cold Air isolated \n Reduce The Noise Outside and Door slamming \n No \n \nSize 93cm * 10cm \n\n 300 ብር \n\nTelegram - t.me/qnashcom\nWebsite - www.qnash.com\n ጥራት ዋስትና ቅናሽ\n አድራሻ መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329\nስልክ:\n0946966440\n0905464599\n +251118639952,Door Bottom Seal Strip Stopper
4,"Washing Machine Anti-Vibration Feet Pads (Shock Pad)\n\n4Packs Anti-Vibration Pads for Washing Machine Home Appliances, Silent Anti-Walk Feet Pads Washer and Dryer Noise Cancelling Foot Pads for All Washing Machine\n\n ዋጋ፦ 350 ብር\n\nውስን ፍሬ ነው የቀረው \n\nTelegram - t.me/qnashcom\nWebsite - www.qnash.com\n ጥራት ዋስትና ቅናሽ\n አድራሻ መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 ከሊፍት እንደወረዱ በግራ በኩል የፎቁ መጨረሻ ዞር እንዳሉ\nስልክ:\n0946966440\n0905464599\n +251118639952",Washing Machine Anti-Vibration Feet Pads (Shock Pad)
5,Helping Handle\n\nHelping Handle with Strong Sucker Hand Grip Handrail to Keep Balance for Bedroom Bath Room Bathroom\n\n ዋጋ፦ 700 ብር\n\nውስን ፍሬ ነው የቀረው \nLimited stock \n\nTelegram - t.me/qnashcom\n ጥራት ዋስትና ቅናሽ\n\n አድራሻ ፦ \nቁጥር 1 :- መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 \n\nቁጥር 2 :- ጀሞ 1 ከለላ ህንፃ ግራውንድ ለይ G07 \n\n ማሳሰብያ: ሱቃችን ሲመጡ ትክክለኛ የኛ ሱቅ መሆኑ ያረጋግጡ የራሳችን ሎጎ መኖሩን እና 329 / G07 መሆኑ ያረጋግጡ !\n\nስልክ:\n 0946966440\n 0992606060\n 0905464599\n +251118639952,Helping Handle
6,ICE Cube Maker Genie\n የበረዶ መስሪያ \n\nKeep Ice-cream cold \nKeep drinks Ice-cold \nEasy to use \nSave a lot of space in your freezer; \nKeeps your cubes fresh and odor free with air tight lid;\n\n ዋጋ፦ 600 ብር \n\nውስን ፍሬ ነው የቀረው \n< Limited Stock \n\nTelegram - t.me/qnashcom\nጥራት ዋስትና ቅናሽ\n\n አድራሻ ፦ \nቁጥር 1 :- መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 \n\nቁጥር 2 :- ጀሞ 1 ከለላ ህንፃ ግራውንድ ለይ G07 \n\n ማሳሰብያ: ሱቃችን ሲመጡ ትክክለኛ የኛ ሱቅ መሆኑ ያረጋግጡ የራሳችን ሎጎ መኖሩን እና 329 / G07 መሆኑ ያረጋግጡ !\n\nስልክ:\n0946966440\n0992606060\n0905464599\n +251118639952,ICE Cube Maker Genie\nየበረዶ መስሪያ
7,Automatic Rebound Healthy Abdomen Wheel\n\nAutomatic Rebound Abdominal Wheel for Core Trainer Workout \n Ab Workout Equipment with Elbow Support \nHome Abdominal Exercise Machine \nand Plank Roller in One\n\n የቦርጭ ስብን ለማቅለጥ እና ፓክ ለማውጣት የሚመረጥ\n በቀላሉ የሆድ፣ የመቀመጫ፣ የጀርባ ጡንቻዎትን ለማሳሳብና ለማጠንከር ይረዳዎታል\n የራሱ መቁጠሪያ ሰዓት የተካተተለት\n በቀላሉ የተሸናቀጠ ተክለሰውነትን ይላበሱ\n\nReliable & Durable\nEasy & Comfortable \nEasy to use \nVast Application \nExcellent Gift Alternative\n\n ዋጋ፦ 2600 ብር \n\n ለማዘዝ = @Orderqnash\n\n ስለ ዕቃወቹ መረጃ እና ቪዲዮዎች ለማየት ቴሌግራማንን ይቀላቀሉ\nTelegram - t.me/qnashcom\n\nውስን ፍሬ ነው የቀረው \nLimited Stock \n ይፍጠኑ \n\nTelegram - t.me/qnashcom\nጥራት ዋስትና ቅናሽ\n\n አድራሻ ፦ \nቁጥር 1 :- መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 \n\nቁጥር 2 :- ጀሞ 1 ከለ

## <font color = '#DF9166' size=6>**Generate CoNLL Label**<font/><a class = 'anchor' id = 'generate_conll'/>

In [21]:
# Messages from '@qnashcom'
qnash_df = df[df["channel_address"] == channel_address].reset_index(drop=True)
qnash_df.head(2)

,channel_name,channel_address,channel_id,message_id,date,message,message_length,word_count,cleaned_message
0,qnash.com - ቅናሽ ®️,@qnashcom,1336267238,18,2020-08-26 05:16:56,😍 ታላቅ ቅናሽ 😍\nዘመናዊ መደርደሪያው ብረት የሆነ ጫማ መደርደሪያ\n\n👟ባለ ዘጠኝ ጫማ መደርደሪያ 👠\n\n➡️ መደርደሪያው ሙሉ በሙሉ ብረት \n➡️ በቀላሉ ሚገጣጠም ሚነቃቀል\n➡️ህይወትዎን ቀለል ሚያረግ ዕቃ\n➡️ቦታ ማይዝ ለቤትዎ ተጨማሪ ውበት ሚሰጥ \n➡️ እስከ 27 ጫማ ማስቀመጥ ሚያስችል\n➡️ለአጠቃቀም በጣም ቀላል \n\nባለ አስር ደረጃ -- 1600 ብር\n \n🏍 ያሉበት ድረስ እናመጣለን\n 0946966440\n 0905464599,293,55,ታላቅ ቅናሽ \nዘመናዊ መደርደሪያው ብረት የሆነ ጫማ መደርደሪያ\n\nባለ ዘጠኝ ጫማ መደርደሪያ \n\n መደርደሪያው ሙሉ በሙሉ ብረት \n በቀላሉ ሚገጣጠም ሚነቃቀል\nህይወትዎን ቀለል ሚያረግ ዕቃ\nቦታ ማይዝ ለቤትዎ ተጨማሪ ውበት ሚሰጥ \n እስከ 27 ጫማ ማስቀመጥ ሚያስችል\nለአጠቃቀም በጣም ቀላል \n\nባለ አስር ደረጃ -- 1600 ብር\n \n ያሉበት ድረስ እናመጣለን\n 0946966440\n 0905464599
1,qnash.com - ቅናሽ ®️,@qnashcom,1336267238,19,2020-08-26 05:32:16,🚫ቤትዎን ከአይጥ 🐀 ከበረሮ🕷 እና ከተባይ🐜 🚫\n ነፃ ያድርጉ\n\n🎯 በቀላሉ በማንኛውም ሶኬት በመሰካት በሚለቀው አልትራሶኒክ ድምፅ በቤት ውስጥ የሚገኙ ጎጅ ነፍሳቶችን ከቤታችን የሚያባርር \n\n🎯 ለቤት ውስጥ ግልጋሎት የሚሆን ምንም የሚረብሽ ድምፅ፣ መጥፎ ሸታ እንዲሁም ኬሚካል የሌለው።\n\n🎯ጥቅም መስጠት የሚያስችለው ስፋት ከ 800_ 1200ካሬ ሜትር እስኩየር \n\n🎯የሀይል ፍጆታው አነስተኛ (90v_250v)\n\n🎯የሚሰራው 24 ሰአት 7 ሳምንት ብሎም ለአመታት \n\n📍ከ 2_3 ሳምንታት በሁዋላ ትክክለኛውን ውጤት ያገኙበታል \nይደውሉ ⤵️\n 📲0946966440 📲0905464599 \nዋጋ 🏷 850ብር \n\n🚛 ያሉበት ድረስ በኛው ትራንሰፓርት 🚚\n በነፃ እናደርሳለን\nለወዳጅዎ #Share ማድረግዎን አይርሱ,473,83,ቤትዎን ከአይጥ ከበረሮ እና ከተባይ \n ነፃ ያድርጉ\n\n በቀላሉ በማንኛውም ሶኬት በመሰካት በሚለቀው አልትራሶኒክ ድምፅ በቤት ውስጥ የሚገኙ ጎጅ ነፍሳቶችን ከቤታችን የሚያባርር \n\n ለቤት ውስጥ ግልጋሎት የሚሆን ምንም የሚረብሽ ድምፅ፣ መጥፎ ሸታ እንዲሁም ኬሚካል የሌለው።\n\nጥቅም መስጠት የሚያስችለው ስፋት ከ 800_ 1200ካሬ ሜትር እስኩየር \n\nየሀይል ፍጆታው አነስተኛ (90v_250v)\n\nየሚሰራው 24 ሰአት 7 ሳምንት ብሎም ለአመታት \n\nከ 2_3 ሳምንታት በሁዋላ ትክክለኛውን ውጤት ያገኙበታል \nይደውሉ \n 0946966440 0905464599 \nዋጋ 850ብር \n\n ያሉበት ድረስ በኛው ትራንሰፓርት \n በነፃ እናደርሳለን\nለወዳጅዎ #Share ማድረግዎን አይርሱ


In [24]:
qnash_df["price"] = qnash_df["cleaned_message"].apply(extract_price)
qnash_df["location"] = qnash_df["cleaned_message"].apply(extract_location)
qnash_df["product"] = qnash_df["cleaned_message"].apply(extract_product)
qnash_df[["cleaned_message", "price", "location", "product"]].sample(5)

,cleaned_message,price,location,product
2575,የልጆች ምሳ ዕቃ ከነውሃ መጠጫ ኮዳ \n\n ዋጋ፦ 1300 ብር\n\nውስን ፍሬ ነው የቀረው \n\nTelegram - t.me/qnashcom\nጥራት ዋስትና ቅናሽ\n\n አድራሻ ፦ \nቁጥር 1 :- መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 \n\nቁጥር 2 :- ጀሞ 1 ከለላ ህንፃ ግራውንድ ለይ G07 \n\n ማሳሰብያ: ሱቃችን ሲመጡ ትክክለኛ የኛ ሱቅ መሆኑ ያረጋግጡ የራሳችን ሎጎ መኖሩን እና 329 / G07 መሆኑ ያረጋግጡ !\n\nስልክ:\n 0946966440\n 0992606060\n 0905464599,ዋጋ፦ 1300 ብር,አድራሻ ፦ ቁጥር 1 :- መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 ቁጥር 2 :- ጀሞ 1 ከለላ ህንፃ ግራውንድ ለይ G07,የልጆች ምሳ ዕቃ ከነውሃ መጠጫ ኮዳ
3021,Make Life Easy \n \n Saachi Hot plate\nሳቺ ባለሁለት ምድጃ ስቶቭ \n\n የማይዝግ ፈሳሽ የማያስገባ \n 2500 ዋት\n ፊውዝ የተገጠመለት\n ትልቅ ድስት መሸከም የሚችል\n አስተማማኝ ቴርሞስታት\n\n Double burner\n Stainless steel body\n Adjustable thermostat\n Die casting heating element\n Automatic safety shut-off\n Non-slip rubber feet\n\n ዋጋ፦ 3800 ብር\n\nውስን ፍሬ ነው የቀረው \n< Limited Stock \n\nTelegram - t.me/qnashcom\nWebsite - www.qnash.com\n ጥራት ዋስትና ቅናሽ\n\n አድራሻ ፦ \nቁጥር 1 :- መገናኛ ዘፍመሽ ግራንድ ሞል 3ተኛ ፎቅ ቁጥር 329 \n\nቁጥር 2 :- ጀሞ 1 ከለላ ህንፃ ግራውንድ ለይ G07 \n\n ማሳሰብያ: ሱቃችን ሲመጡ ትክክለኛ የኛ ሱቅ መሆኑ ያረጋግጡ የራሳችን ሎጎ መኖሩን እና 329 / G07 መሆኑ ያረጋግጡ !\n\nስልክ:\n0946966440\n0992606060\n0905464599,ዋጋ፦ 3800 ብር,አድራሻ ፦ ቁጥር 1 :- መገናኛ ዘፍመሽ ግራንድ ሞል 3ተኛ ፎቅ ቁጥር 329 ቁጥር 2 :- ጀሞ 1 ከለላ ህንፃ ግራውንድ ለይ G07,Make Life Easy\n
2625,"Hair Dye Brush Bottle\n\nMade of quality plastic material\nGreat tool for hair dyeing use\nEasy to mix hair color potion\nSuitable for both professional or home DIY use\nAttached with a comb, durable and safe to use\nBottle with scale, easy to mix hair color potio\n\n ዋጋ፦ 200 ብር \n\nውስን ፍሬ ነው የቀረው \n< Limited Stock \n\nTelegram - t.me/qnashcom\n ጥራት ዋስትና ቅናሽ\n\n አድራሻ ፦ \nቁጥር 1 :- መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 \n\nቁጥር 2 :- ጀሞ 1 ከለላ ህንፃ ግራውንድ ለይ G07 \n\n ማሳሰብያ: ሱቃችን ሲመጡ ትክክለኛ የኛ ሱቅ መሆኑ ያረጋግጡ የራሳችን ሎጎ መኖሩን እና 329 / G07 መሆኑ ያረጋግጡ !\n\nስልክ:\n0946966440\n0992606060\n0905464599",ዋጋ፦ 200 ብር,አድራሻ ፦ ቁጥር 1 :- መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 ቁጥር 2 :- ጀሞ 1 ከለላ ህንፃ ግራውንድ ለይ G07,Hair Dye Brush Bottle
1378,Helping Handle\n\nHelping Handle with Strong Sucker Hand Grip Handrail to Keep Balance for Bedroom Bath Room Bathroom\n\n ዋጋ፦ 700 ብር\n\nውስን ፍሬ ነው የቀረው \nLimited stock \n\nTelegram - t.me/qnashcom\n ጥራት ዋስትና ቅናሽ\n\n አድራሻ ፦ \nቁጥር 1 :- መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 \n\nቁጥር 2 :- ጀሞ 1 ከለላ ህንፃ ግራውንድ ለይ G07 \n\n ማሳሰብያ: ሱቃችን ሲመጡ ትክክለኛ የኛ ሱቅ መሆኑ ያረጋግጡ የራሳችን ሎጎ መኖሩን እና 329 / G07 መሆኑ ያረጋግጡ !\n\nስልክ:\n 0946966440\n 0992606060\n 0905464599\n +251118639952,ዋጋ፦ 700 ብር,አድራሻ ፦ ቁጥር 1 :- መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 ቁጥር 2 :- ጀሞ 1 ከለላ ህንፃ ግራውንድ ለይ G07,Helping Handle
2106,5 in 1 Beauty Care Massager\n\nለአጠቃቀም ቀላል ውበትን የሚያላበስ አዲስ ዘመናዊ የፊት ማፅጃ \n\nለመታጠቢያ ሁነኛ ተመራጭ\nየሞተ ቆዳ የሚያነሳ\nየቆዳ መሸብሸብን ይከላከላል\nለስላሳ እና ውብ ፊት ያላብሳል \nየደም ዝውውርን ውጤታማ በሆነ መንገድ የሚያፋጥን \n እንዲሁም የተዘጋ ቀዳዳዎችን ፣ ዘይትን ፣ እና ሌሎች አላስፈላጊ ነገሮችን በቀላሉ ያስወግዳል \nለሁሉም ፆታ የሚሆን \nበባትሪ መስራቱ ከቦታ ቦታ ይዞ ለመንቀሳቀስ ተመራጭ ያደርገዋል \n\n#አራት_ተቀያያሪ_ብሩሾች_ያሉት \nየሞተ ቆዳ ማራገፊያ ብሩሽ \nሜካፕ ስፖንጅ ብሩሽ\nየፊት ማለስለሻ ብሩሽ\nማሳጅ ማድረጊያ ብሩሽ\nየፊት ቆዳ ማፅጃ ብሩሽ\n\n ዋጋ፦ 750 ብር\n\nውስን ፍሬ ነው የቀረው \n< Limited Stock \n\nTelegram - t.me/qnashcom\nTelegram - t.me/qnashcom\nጥራት ዋስትና ቅናሽ\n\n አድራሻ ፦ \nቁጥር 1 :- መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 \n\nቁጥር 2 :- ጀሞ 1 ከለላ ህንፃ ግራውንድ ለይ G07 \n\n ማሳሰብያ: ሱቃችን ሲመጡ ትክክለኛ የኛ ሱቅ መሆኑ ያረጋግጡ የራሳችን ሎጎ መኖሩን እና 329 / G07 መሆኑ ያረጋግጡ !\n\nስልክ:\n0946966440\n0992606060\n0905464599\n +251114622244,ዋጋ፦ 750 ብር,አድራሻ ፦ ቁጥር 1 :- መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 ቁጥር 2 :- ጀሞ 1 ከለላ ህንፃ ግራውንድ ለይ G07,5 in 1 Beauty Care Massager


### Load the CoNLL File

In [25]:
label_data_conll(qnash_df, "../data/processed/labeled_data.conll")

### Load the CoNLL File

In [35]:
conll_df = load_conll_to_dataframe("../data/processed/labeled_data.conll")
conll_df.head()

,message_num,token,label
0,1,Wonder,B-PRODUCT
1,1,Korean,I-PRODUCT
2,1,Product,I-PRODUCT
3,1,P.L.C,I-PRODUCT
4,1,Wonder,B-PRODUCT


In [36]:
print(f"Number of unique message_nums: {conll_df['message_num'].nunique()}")

Number of unique message_nums: 2313


A total of 2,313 messages were automatically labeled using this method.